# Neutron Powder Diffraction

In this tutorial demonstrates how neutron-scattering data can be loaded, visualized, and manipulated with generic functionality from `scipp` as well as neutron-specific functionality from `scipp.neutron`. It focuses on reducing data from the ORNL [POWGEN](https://neutrons.ornl.gov/powgen) neutron diffractometer.

In [ ]:
import numpy as np
import scipp as sc

### Loading Nexus files

Loading Nexus files requires [Mantid](https://www.mantidproject.org).
See, e.g., [Installation](https://scipp.github.io/getting-started/installation.html) on how to install scipp and Mantid with `conda`.
We are using two files in this tutorial,
[PG3_4844_event.nxs](http://198.74.56.37/ftp/external-data/MD5/d5ae38871d0a09a28ae01f85d969de1e)
and
[PG3_4866_event.nxs](http://198.74.56.37/ftp/external-data/MD5/3d543bc6a646e622b3f4542bc3435e7e).
Both are available as part of Mantid's test data.

Rename the files upon download.

We start by loading two files: the sample and the vanadium runs.

In [ ]:
sample = sc.neutron.load(filename='PG3_4844_event.nxs',
                         load_pulse_times=False,
                         mantid_args={'LoadMonitors': True})
vanadium = sc.neutron.load(filename='PG3_4866_event.nxs',
                           load_pulse_times=False)

The optional `mantid_args` dict is forwarded to the Mantid algorithm used for loading the files &ndash; in this case [LoadEventNexus](https://docs.mantidproject.org/nightly/algorithms/LoadEventNexus-v1.html) &ndash; and can be used to control, e.g., which part of a file to load.
Here we request loading monitors, which Mantid does not load by default.
The resulting dataset looks as follows:

In [ ]:
sample

In [ ]:
sc.plot(sample)

### Instrument view

Scipp provides a simple 3D instrument view inpired by Mantid's own [instrument view](https://www.mantidproject.org/MantidPlot:_Instrument_View), which can be used to take a quick look at the neutron counts on the detector panels in 3D space or using various cylindrical and spherical projections

<div class="alert alert-info">

**Note**

The support and handling of event data is currently being rewritten.
The interface for manipulating event data (bucketed) data is not representative for the final API.

</div>

In [ ]:
sc.neutron.instrument_view(sample)

### Plot against scattering angle $\theta$ using `groupby`

*This is not an essential step and can be skipped.*

Plotting raw data directly yields a hard-to-interpret figure.
We can obtain something more useful by "binning" the spectrum axis based on its $\theta$ value, using the split-apply-combine approach provided by `groupby`:

In [ ]:
sample.coords['theta'] = sc.neutron.scattering_angle(sample)
vanadium.coords['theta'] = sc.neutron.scattering_angle(vanadium)
theta_bins = sc.Variable(['theta'],
                         unit=sc.units.rad,
                         values=np.linspace(0.0, np.pi/2, num=2000))

We concatenate events lists from different spectra that fall into a given theta range into longer combined lists:

In [ ]:
theta_sample = sc.groupby(sample, 'theta', bins=theta_bins).bins.concatenate('spectrum')

<div class="alert alert-info">

**Note**
    
Use `groupby.sum` instead of `groupby.bins.concatenate` when working with dense (histogrammed) data
</div>

In [ ]:
theta_sample

In [ ]:
sc.plot(theta_sample)

### Unit conversion

*Note: We are back to working with `sample`, not `theta_sample`.*

`scipp.neutron` provides means to convert between units (dimensions) related to time-of-flight.
The loaded raw data has `Dim.Tof`, and we convert to interplanar lattice spacing (d-spacing):

In [ ]:
dspacing_vanadium = sc.neutron.convert(vanadium, 'tof', 'd-spacing', out=vanadium)
dspacing_sample = sc.neutron.convert(sample, 'tof', 'd-spacing')
dspacing_sample

### Neutron monitors

*This is an optional section.
The next section does not use the monitor-normalized data produced here.
This section could thus be skipped.*

If available, neutron monitors are stored as attributes of a data array:

In [ ]:
mon = sample.attrs['monitor1'].value
mon

The monitor could, e.g., be used to normalize the data.
To do so, both data and monitor need to be converted to a unit that accounts for differing flight paths, e.g., wavelength or energy:

In [ ]:
sample_lambda = sc.neutron.convert(sample, 'tof', 'wavelength')
mon = sc.neutron.convert(mon, 'tof', 'wavelength')

The sample data is in event-mode, i.e., is not histogrammed.
Event data *can* be divided by a histogram (such as `mon` in this case), using a specialized function for scaling (see [Bucketed data](../user-guide/binned-data.rst)).
First we rebin the monitor since the original binning is very fine:

In [ ]:
edges = sc.Variable(dims=['wavelength'], unit=sc.units.angstrom, values=np.linspace(0, 1, num=1000))
mon = sc.rebin(mon, 'wavelength', edges)
mon

The sample data is *event data in bins* and the monitor is a histogram.
Multiplication and division operations for such cases are supported by modifying the weights (values) for each event using the operators of the `bins` property, in combination with the `sc.lookup` helper, a wrapper for a discrete "function", given by the monitor:

In [ ]:
sample_over_mon = sample_lambda.bins / sc.lookup(func=mon, dim='wavelength')
sample_over_mon

Finally, we can plot the event data with on-the-fly binning.
By default, the `plot` function uses the coordinates of the binned data to define histogram edges, which, in this case, would give a single bin along the `'tof'` dimension.
For a better representation of the data, we supply the bin edges to obtain a finer binning, yielding a more meaningful figure:

In [ ]:
sc.plot(sample_over_mon)

In [ ]:
del sample_lambda
del sample_over_mon
del sample
del vanadium

### From events to histogram

*Note: We are continuing here with data that has not been normalized to the monitors.*

We histogram the event data:

In [ ]:
dspacing_bins = sc.Variable(
    ['d-spacing'],
    values=np.arange(0.3, 2.0, 0.001),
    unit=sc.units.angstrom)
hist = sc.Dataset({'sample':sc.histogram(dspacing_sample, dspacing_bins),
                   'vanadium':sc.histogram(dspacing_vanadium, dspacing_bins)})
sc.show(hist['spectrum', 0:3]['d-spacing', 0:7])

In [ ]:
sc.plot(hist)

### Summing (focussing) and normalizing

After conversion to `'d-spacing'`, generic `sum` and `/` operations can be used to "focus" and normalize the diffraction data to the vanadium run:

In [ ]:
summed = sc.sum(hist, 'spectrum')
sc.plot(summed)

In [ ]:
normalized = summed['sample'] / summed['vanadium']
sc.plot(normalized)

### Focussing with $\theta$ dependence in event-mode

The approach used above combines reflections from all crystallographic planes and is therfore of limited use.
We can use `groupby` to focus each of multiple groups of spectra into a distinct output spectrum.
Here we define groups based on a range of scattering angles &ndash; a simple $\theta$-dependent binning.
This also demonstrates how we can postpone histogramming until after the focussing step.

In [ ]:
theta = sc.Variable(['theta'],
                    unit=sc.units.rad,
                    values=np.linspace(0.0, np.pi/2, num=16))

focussed_sample = sc.groupby(dspacing_sample, 'theta', bins=theta).bins.concatenate('spectrum')
focussed_vanadium = sc.groupby(dspacing_vanadium, 'theta', bins=theta).bins.concatenate('spectrum')
norm = sc.histogram(focussed_vanadium, dspacing_bins)
focussed_sample.bins /= sc.lookup(func=norm, dim='d-spacing')
normalized = sc.histogram(focussed_sample, dspacing_bins)

The normalized output looks as follows:

In [ ]:
sc.plot(normalized, vmin=0, vmax=2)

As a bonus, we can use slicing and a dict-comprehension to quickly create of plot comparing the spectra for different scattering angle bins:

In [ ]:
# compute centers of theta bins
angles = normalized.coords['theta'].values
angles = 0.5*(angles[1:] + angles[:-1])
sc.plot({f'{round(angles[group], 3)} rad':
         normalized['d-spacing', 300:500]['theta', group]
         for group in range(2,6)})